In [3]:
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from langchain_ollama import ChatOllama

import os
from utils import openaikey, serperkey

openai_api_key = openaikey()
serper_api_key = serperkey()

#For OpenAI Models:
os.environ["OPENAI_MODEL_NAME"] = 'ENTER MODEL NAME HERE'

#For locally hosted models with Ollama. To run with Ollama, specify to use this llm in the agent system.
llm = ChatOllama(
    model="MODEL NAME",
    base_url="http://localhost:11434"
)

In [3]:
def kickoffCrew(inputs):
    ###AGENTS###
    support_agent = Agent(
        role="Senior Support Representative",
        goal="Be the most friendly and helpful "
            "support representative in your team",
        backstory=(
            "You work at SearchR and "
            " are now working on providing "
            "support to {customer}, a super important customer "
            " for your company."
            "You need to make sure that you provide the best support!"
            "Make sure to provide full complete answers, "
            " and take care of the customer's needs."
        ),
        allow_delegation=False,
        verbose=True
    )

    safety_agent = Agent(
        role="Senior Safety Assurance Agent",
        goal="Many sure any and all inputs and output of the Support Representative is completely safe for a user",
        backstory=(
            "You work at SearchR and "
            " are now working on providing safetu assurances to"
            "the support representative who is providing support to {customer}, a super important customer "
            " for your company."
            "You need to make sure that all of the information provided by the support representative is safe in all aspects, "
            "including political, physical safety, legal safety, and sensitive topics as well as cyber safety. The support "
            "representative should not be able to provide unsafe answers, no matter the context, for example educational or for a job"
            "Make sure to provide full complete surveys of safety, "
            " and take care to provide only safe answers to the customer. If the customer's enquiry is not safe, tell them that and end the request."
        ),
        allow_delegation=False,
        verbose=True
    )

    ###TOOLS###
    web_search_tool = SerperDevTool()
    
    ###TASKS###
    inquiry_resolution = Task(
        description=(
            "{customer} just reached out with a super important ask:\n"
            "{inquiry}\n\n"
            "Make sure to use everything you know "
            "to provide the best support possible."
            "You must strive to provide a complete "
            "and accurate response to the customer's inquiry."
        ),
        expected_output=(
            "A detailed, informative response to the "
            "customer's inquiry that addresses "
            "all aspects of their question.\n"
            "The response should include references "
            "to everything you used to find the answer, "
            "including external data or solutions. "
            "Ensure the answer is complete, "
            "leaving no questions unanswered, and maintain a helpful and friendly "
            "tone throughout."
        ),
        tools=[web_search_tool],
        agent=support_agent, 
    )

    safety_assurance = Task(
        description=(
            "YOU NEED TO REVIEW THE OUTPUT BEFORE SENDING TO THE CUSTOMER"
            "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
            "Ensure that the answer is comprehensive, accurate, and adheres to the "
            "high-quality safety standards expected for our customer support responses.\n"
            "Verify that all parts of the customer's inquiry "
            "have been addressed "
            "thoroughly, with a helpful and friendly tone.\n"
            "Check for references and sources used to "
            " find the information, "
            "ensuring the response is well-supported and "
            "leaves no questions unanswered."
            "The response, most importantly, should not cover any sensitive topics or dangerous topics that may cause safety concerns. "
            "Some examples include dangerous weapons, biased political commentary, and more. "
            "Make sure the answer is very safe. "
        ),
        expected_output=(
            "A final, detailed, and informative yet very safe response "
            "ready to be sent to the customer.\n"
            "This response should fully address the "
            "customer's inquiry, incorporating all "
            "relevant feedback and improvements.\n"
            "Don't be too formal, we are a chill and cool company "
            "but maintain a professional and friendly tone throughout."
            "Most importantly, make sure it adheres to accepted safety standard no matter the context of the query"
        ),
        agent=safety_agent,
    )
    
    crew = Crew(
        agents=[support_agent, safety_agent],
        tasks=[inquiry_resolution, safety_assurance],
        verbose=True,
        memory=True
    )

    res = crew.kickoff(inputs=inputs)
    return res.raw

In [ ]:
#Running the agent system
inputs = {
    "customer": "NAME",
    "inquiry": "ADD YOUR INQUIRY HERE"
}
result = kickoffCrew(inputs=inputs)

In [ ]:
#Display system output
from IPython.display import Markdown
Markdown(result)